# Basic SQL Database Example

Relational databases are a common way to store data. They are used to store data in a structured manner, allowing for efficient data retrieval and manipulation. SQL (Structured Query Language) is a standard language for relational databases and used in many applications. 

In this example, we will see how to use mdmodels to interact with a SQL database. We will start by creating a database and tables from a markdown file, insert data into the database, update and delete rows and also restore nested objects.

In [1]:
import json
import rich

from mdmodels import DataModel
from mdmodels import sql

## Setup and creation of the database

We start by creating the data model from the markdown file and then generate the SQLModel classes. In the background will align the database data model with the hierarchical data model defined in the markdown file.

Once we have the SQLModel classes, we can create the database and the tables by using the `DatabaseConnector` class. The latter creates a so called `engine` which is used to connect to the database. This `engine` is then used to create the tables. Later on we will make use of this `engine` to interact with the database.

In [2]:
# Create the data model from the markdown file
dm = DataModel.from_markdown("model.md")

# Generate the SQLModel classes
models = sql.generate_sqlmodel(data_model=dm)

# We use an in-memory database for this example
# Leaving the database empty, will automatically create it
db = sql.DatabaseConnector(database="")
db.create_tables(models)


## Creating a row and inserting it into the database

In the following example, we will create a `Molecule` row and insert it into the database. Since we are using [SQLModel](https://sqlmodel.tiangolo.com/), which is a library that extends the SQLAlchemy library, we first need to start a session.

```python
with db as session:
    # Do something with the session
```

Sessions are context managers and take care of committing and rolling back transactions to the database. Once exited, the session is closed and the transaction is committed. If there is an exception, the transaction is rolled back. This gives you a safe way to interact with the database.

Moving on, creating a row behaves the same way as in our basic example. You provide the necessary information as attributes of the SQLModel class and add it. Lets do it!

In [3]:
with db as session:
    # Create a molecule
    molecule = models.Molecule(id="123", name="Methanol", smiles="CO")

    # Now we add the molecule to the session
    session.add(molecule)

### Querying the database

Now that we have added our first row, we'd like to check if it was added correctly. Instead of creating a new row, we now want to `select` row(s) from the database. Here's an explanation of how to do this:

- `select` initializes a query that selects something from the database.
- We pass the SQLModel class we want to select from.
- Using `session.exec()` we execute the query.
- `.all()` returns all the results of the query as a list.

Let's see how this works in practice!


In [4]:
with db as session:
    statement = sql.select(models.Molecule)
    molecules = session.exec(statement).all()

    rich.print(molecules)

[Molecule(id='123', formula=None, name='Methanol')]

Great! We have successfully created a row and queried it from the database. For the sake of completeness, lets explore how we can `update` and `delete` rows from the database.

### Updating a row

Updating a row is done by first selecting the row you want to update and then changing the attributes you want to update. After that, you simply add the updated row to the session and it will be automatically commited to the database.

### Deleting a row

Deleting a row is done by first selecting the row you want to delete and then deleting it from the session.

In [5]:
# Updating a row
with db as session:
    statement = sql.select(models.Molecule).where(models.Molecule.id == "123")
    molecule = session.exec(statement).first()

    molecule.name = "Methanol (Updated)"
    session.add(molecule)

    session.commit() # You can also commit manually
    session.reset() # Resets the session to its initial state

    # Verify the update
    statement = sql.select(models.Molecule).where(models.Molecule.id == "123")
    molecule = session.exec(statement).first()

    rich.print(molecule) # Should now print the updated molecule

Molecule(id='123', formula=None, name='Methanol (Updated)')

In [6]:
# Deleting a row
with db as session:
    statement = sql.select(models.Molecule).where(models.Molecule.id == "123")
    molecule = session.exec(statement).first()

    session.delete(molecule)
    session.commit()
    session.reset()

    # Verify the deletion
    statement = sql.select(models.Molecule).where(models.Molecule.id == "123")
    molecule = session.exec(statement).first()

    rich.print(molecule) # Should now print None

None

## Inserting nested objects

Okay, so we have seen how to create, update and delete rows from the database. But what if we have nested objects? Typically, when using mdmodels, you will have nested objects. Remember, we want to bridge the format and database word and in the former, we usually deal with nested objects.

Adding nested objects is a complicated process, especially when you want to insert multiples of them. Data Races, where one row might be inserted before another, are a common problem. To solve this, we have the `insert_nested` function that takes care of this for you.

### How it works

The `insert_nested` function recursively and asynchronously adds the rows to the __session__. If it encounters an object which is already in the session, it will simply re-use it. This way we can ensure that our database is non-redundant and that we don't have any data races.

Lets see how this works in practice!

In [7]:
# First, we load a couple of objects from a JSON file
# Feel free to inspect the file to see what it contains - pretty nested!
with open("projects.json", "r") as f:
    datasets = [dm.ChemicalProject(**dataset) for dataset in json.load(f)]

In [8]:
# Same stuff as before, we start a session
with db as session:
    # Next, we use the `insert_nested` function to add the rows to the session
    to_add = sql.insert_nested(datasets, dm, session, models)

    # Finally, we add the rows to the session
    session.add_all(to_add)
    session.commit()

    # Now we can verify that the rows were added correctly
    statement = sql.select(models.ChemicalProject)
    projects = session.exec(statement).all()

    rich.print(projects)

[
    ChemicalProject(title='Green Energy Catalysis'),
    ChemicalProject(title='Ethanol Studies'),
    ChemicalProject(title='Hydrogen Research')
]

## Restoring nested objects

Adding nested objects is one thing, but what if we want to restore them from the database? This is also possible with mdmodels!

Typically, when you query a row from the database, the nested objects are not restored. This is called a "lazy load". If you want to restore the nested objects, you can do so by using the `to_dict` function. This function will recursively restore the nested objects from the database.

_Why into a dictionary?_ Well, we dont want to impose the `DataModel` class to you, since you might want to do other things with it and a dictionary is more flexible in terms of types. If you prefer to use the `DataModel`, we will explore how to do this very easily in the next example.


In [9]:
with db as session:
    statement = sql.select(models.ChemicalProject)
    project = session.exec(statement).first()

    # Lets get the dictionary representation of the project
    project_dict = project.to_dict()

    # ... and restore it using the DataModel class
    project = dm.ChemicalProject(**project_dict)

    rich.print(project)

ChemicalProject(
    title='Green Energy Catalysis',
    authors=[Author(orcid='0000-0001-2345-6789', name='Dr. Alice Green')],
    molecules=[
        Molecule(id='M001', name='Water', formula='H2O'),
        Molecule(id='M003', name='Oxygen', formula='O2')
    ],
    reactions=[
        Reaction(
            id='R002',
            name='Water Formation',
            educts=[Element(molecule_id='M003', stoichiometry=2.0)],
            products=[Element(molecule_id='M001', stoichiometry=2.0)]
        )
    ],
    experiments=[
        Experiment(
            id='E002',
            initial_concentrations=[Concentration(molecule_id='M003', value=1.5, unit='mol/L')]
        )
    ],
    kinetic_models=[KineticModel(molecule_id='M003', equation='rate = k * [O2]^2')]
)

## Advanced queries

Cool, we have seen how to create, update, delete and restore rows from the database. We have mostly obeyed the strict hierarchy of our mode, but as you might recall looking into [`model.md`](model.md), you may have noticed the following:

```markdown
### Concentration

- molecule_id
    - Type: string
    - References: ChemicalProject.molecules.id
```

This is a many-to-one relationship, meaning that a `Molecule` can have multiple `Concentration`s, but a `Concentration` can only have one `Molecule`. These cross-references are automatically generated for you when you use the `generate_sqlmodel` function.

What if we want to find all projects where molecule `Ethanol` appears as an educt? We can do this by using the `select` function and the `join` method.

### What is `join`?

The `join` method is used to join two tables together. In this case, we want to join the `ChemicalProject` table with the `Concentration` table on the `molecule_id` column. This is exactly what the `join` method does!

Lets see how this works in practice!


In [10]:
with db as session:
    # JOIN the tables on the molecule_id column
    statement = (
        sql.select(models.ChemicalProject, models.Molecule)
           .join(models.Reaction, models.ChemicalProject.reactions)
           .join(models.Element, models.Reaction.educts)
           .join(models.Molecule, models.Element.molecule_id == models.Molecule.id)
           .where(models.Molecule.name.like("%Ethanol%"))
    )   

    rows = session.exec(statement).all()
    
    for project, molecule in rows:
        rich.print(
            f"[bold cyan]{molecule.name}[/bold cyan] ({molecule.id}) is educt "
            f"in [bold green]{project.title}[/bold green]"
        )


Ethanol (M002) is educt in Ethanol Studies

## Conclusion

In this example, we have seen how to create, update, delete and restore rows and nested objects from the database. We have also seen how to use the `join` method to perform advanced queries. This should give you a good starting point for working with mdmodels and SQL databases.